In [51]:
from dataclasses import dataclass
from typing import  Any
from enum import Enum


In [52]:

class Color(Enum):
    RED = True
    BLACK = False

In [68]:
@dataclass
class Node:
    key: Any
    val: Any
    color: Color
    parent: object
    left: object
    right: object

    def __str__(self) -> str:
        return f'\t\tKey: {self.key}; Val: {self.val}; Parent:{self.parent.key if self.parent is not None else self.parent}\n Left:{self.left.key}\t\t\t Right:{self.right.key}'

    def grand_parent(self):
        if self.parent is not None:
            return self.parent.parent
        else:
            return None

    def uncle(self):
        gp = self.grand_parent()
        if gp is None:
            return None
        if self.parent is gp.left:
            return gp.right
        else:
            return gp.left
        
    def sibling(self):
        if self.parent is None:
            return None
        if self is self.parent.left:
            return self.parent.right
        else:
            return self.parent.left

In [69]:
class rb_tree:
    root: object

    # Temporary constructor
    def __init__(self, root=None) -> None:
        self.root = root

    def rotate_left(self, node):
        # We are given the parent of the new node
        gp = node.parent
        # We change the  child of grand parent to the new node
        if node is gp.left:
            gp.left = node.right
            gp.left.parent = gp
        else:
            gp.right = node.right
            gp.right.parent = gp

        # We will change the value of the right child of the parent to the left child of the new node
        temp_right_node = node.right
        node.right = node.right.left
        node.right.parent = node

        temp_right_node.left = node
        node.parent = temp_right_node


    def insert(self, key, val):
        # Create a red node with two black leaves
        node = Node(key, val, Color.RED, None, None, None)
        node.left = Node(None, None, Color.BLACK, node, None, None)
        node.right = Node(None, None, Color.BLACK, node, None, None)
        # Insert the node at a leaf position
        if self.root is None:
            position_node = node
            self.root = position_node
        else:
            position_node = self.search_leaf(self.root, key)
            # Since we don't know if the node we got is the left or right child, we compare
            if position_node.parent.left is position_node:
                position_node.parent.left = node
            else:
                position_node.parent.right = node

        # Balancing the tree after insert operation
        # Case 1
        if position_node.parent is None:
            position_node.color = Color.BLACK

        # Case 2
        elif position_node.parent.color == Color.BLACK:
            pass

    # Reccursive function to insert a node like a binary search tree
    def search_leaf(self, node, search_key) -> Node:
        # Since in rb trees we also insert None nodes after each node, we need to
        # check if key of the node is none
        if node.key is None:
            return node

        if node.key < search_key:
            return self.search_leaf(node.right, search_key)

        return self.search_leaf(node.left, search_key)

In [72]:
# A valid Red-Black Tree
# Children of right node

# Node(key, val, color, parent, left, right)
right_l = Node(3, None, Color.RED, None, None, None)
right_l.left = Node(None, None, Color.BLACK, right_l, None, None)
right_l.right = Node(None, None, Color.BLACK, right_l, None, None)

right_r = Node(5, None, Color.RED, None, None, None)
right_r.left = Node(None, None, Color.BLACK, right_r, None, None)
right_r.right = Node(None, None, Color.BLACK, right_r, None, None)
# Root Node
root = Node(2, None, Color.BLACK, None, None, None)

right = Node(4, None, Color.BLACK, root, right_l, right_r)
left = Node(1, None, Color.BLACK, root, None, None)
# Children of root
root.left = left
root.right = right

right_r.parent = right
right_l.parent = right

left.left= Node(None, None, Color.BLACK, left, None, None)
left.right = Node(None, None, Color.BLACK, left, None, None)

# Pre constructed tree example
rb = rb_tree(root)



print("Before Rotation")
print("Root")
print(str(root))
print("Root.right")
print(str(root.right))
print("Root.right.right")
print(str(root.right.right))

print('\n\nAfter Rotation')
rb.rotate_left(right)
print("Root")
print(str(root))
print("Root.right")
print(str(root.right))
print("Root.right.left")
print(str(root.right.left))



Before Rotation
Root
		Key: 2; Val: None; Parent:None
 Left:1			 Right:4
Root.right
		Key: 4; Val: None; Parent:2
 Left:3			 Right:5
Root.right.right
		Key: 5; Val: None; Parent:4
 Left:None			 Right:None


After Rotation
Root
		Key: 2; Val: None; Parent:None
 Left:1			 Right:5
Root.right
		Key: 5; Val: None; Parent:2
 Left:4			 Right:None
Root.right.left
		Key: 4; Val: None; Parent:5
 Left:3			 Right:None


In [66]:
# Empty Tree example

rb = rb_tree()
rb.insert(1, 2)
rb.insert(10,2)
print(rb.root)

		Key: 1; Val: 2; Parent:None
 Left:None			 Right:10
